In [2]:
import pandas as pd
import numpy as np
import json
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import dateutil.parser as parser
from datetime import datetime, date, timedelta
import torch
import skorch
import scipy
import torch.nn as nn
import torch
import torch.nn.functional as F
import sys
from skorch.helper import DataFrameTransformer
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error, r2_score
from sklearn import metrics
import math

In [22]:
# Opening JSON file
f = open('twitter_aapl_new.json',)
  
# returns JSON object as 
# a dictionary
aapl = json.load(f)

f.close()

In [23]:
flataapl  = [val for sublist in aapl for val in sublist]

In [24]:
print(flataapl[0])

{'text': 'RT @DomainFQ: .\nDomain Name for sale\n\nhttps://t.co/f056WpEdXS\n\n#fluke #FlukeTV #domain #100DaysOfCode #javascript #python #Apple #vr #Micro…', 'referenced_tweets': [{'type': 'retweeted', 'id': '1391353131334717440'}], 'id': '1392083055460851716', 'lang': 'en', 'created_at': '2021-05-11T11:43:58.000Z'}


In [25]:
df = pd.DataFrame(flataapl)
df

,text,referenced_tweets,id,lang,created_at,withheld
0,RT @DomainFQ: .\nDomain Name for sale\n\nhttps...,"[{'type': 'retweeted', 'id': '1391353131334717...",1392083055460851716,en,2021-05-11T11:43:58.000Z,NaN
1,RT @DomainFQ: .\nDomain NameFor Sale\n\nhttps:...,"[{'type': 'retweeted', 'id': '1391353688401186...",1392083046459969541,en,2021-05-11T11:43:56.000Z,NaN
2,RT @DomainFQ: .\nDomains For sale\n\nhttps://t...,"[{'type': 'retweeted', 'id': '1391353349669154...",1392083045058940935,en,2021-05-11T11:43:56.000Z,NaN
3,RT @DomainFQ: .\nDomain NameFor Sale\n\nhttps:...,"[{'type': 'retweeted', 'id': '1391353688401186...",1392083027929358336,en,2021-05-11T11:43:52.000Z,NaN
4,🍎 Apple Air Tags\nCALL FOR PRICE\nMOQ APPLY\nC...,NaN,1392083005594873856,en,2021-05-11T11:43:46.000Z,NaN
...,...,...,...,...,...,...
3390449,Silicone Case Cover Shockproof Holder For Appl...,NaN,1259830641736273921,en,2020-05-11T13:00:06.000Z,NaN
3390450,#Apple has shown that #iPhone sales fell more ...,NaN,1259830629723766785,en,2020-05-11T13:00:03.000Z,NaN
3390451,Check it OUT Shadowland by C.M.Gray Read the F...,NaN,1259830628721229826,en,2020-05-11T13:00:03.000Z,NaN
3390452,Le casque circum-aural d’Apple serait baptisé ...,NaN,1259830627270119427,fr,2020-05-11T13:00:02.000Z,NaN


In [26]:
y = df['withheld'].dropna()
y

111581     {'copyright': False, 'country_codes': ['IN']}
169202     {'copyright': False, 'country_codes': ['IN']}
199596     {'copyright': False, 'country_codes': ['IN']}
304544     {'copyright': False, 'country_codes': ['IN']}
317803     {'copyright': False, 'country_codes': ['IN']}
                               ...                      
3264756    {'copyright': False, 'country_codes': ['IN']}
3264931    {'copyright': False, 'country_codes': ['IN']}
3265032    {'copyright': False, 'country_codes': ['IN']}
3265033    {'copyright': False, 'country_codes': ['IN']}
3352769    {'copyright': False, 'country_codes': ['IN']}
Name: withheld, Length: 125, dtype: object

In [27]:
df1 = df.drop('withheld',axis=1)
df1['created_at'] = df1['created_at'].map(lambda x: str(x)[:-5])
df1

,text,referenced_tweets,id,lang,created_at
0,RT @DomainFQ: .\nDomain Name for sale\n\nhttps...,"[{'type': 'retweeted', 'id': '1391353131334717...",1392083055460851716,en,2021-05-11T11:43:58
1,RT @DomainFQ: .\nDomain NameFor Sale\n\nhttps:...,"[{'type': 'retweeted', 'id': '1391353688401186...",1392083046459969541,en,2021-05-11T11:43:56
2,RT @DomainFQ: .\nDomains For sale\n\nhttps://t...,"[{'type': 'retweeted', 'id': '1391353349669154...",1392083045058940935,en,2021-05-11T11:43:56
3,RT @DomainFQ: .\nDomain NameFor Sale\n\nhttps:...,"[{'type': 'retweeted', 'id': '1391353688401186...",1392083027929358336,en,2021-05-11T11:43:52
4,🍎 Apple Air Tags\nCALL FOR PRICE\nMOQ APPLY\nC...,NaN,1392083005594873856,en,2021-05-11T11:43:46
...,...,...,...,...,...
3390449,Silicone Case Cover Shockproof Holder For Appl...,NaN,1259830641736273921,en,2020-05-11T13:00:06
3390450,#Apple has shown that #iPhone sales fell more ...,NaN,1259830629723766785,en,2020-05-11T13:00:03
3390451,Check it OUT Shadowland by C.M.Gray Read the F...,NaN,1259830628721229826,en,2020-05-11T13:00:03
3390452,Le casque circum-aural d’Apple serait baptisé ...,NaN,1259830627270119427,fr,2020-05-11T13:00:02


In [8]:
print(type(df1['referenced_tweets'][3390453]))
print(isinstance(df1['referenced_tweets'][4], float))

<class 'float'>
True


In [28]:
dfsmall = df1.iloc[0:10]
dfsmall
df2 = df1

In [ ]:
d = df1.loc[df1['text'] == str("It's no secret that Apple products are pricey")]
g = df1.loc[df1['id'] == str(1259825816600444929)]
print(g)
g.iloc[0,0]
df1['id'].isnull().values.any()

In [29]:
dfcut = df2
dfcut = dfcut.dropna(subset=['referenced_tweets'])
dfcut['referenced_tweets'] = dfcut['referenced_tweets'].map(lambda x: x[0]['id'])
dfcut

<ipython-input-29-400ff7227f34>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfcut['referenced_tweets'] = dfcut['referenced_tweets'].map(lambda x: x[0]['id'])


,text,referenced_tweets,id,lang,created_at
0,RT @DomainFQ: .\nDomain Name for sale\n\nhttps...,1391353131334717440,1392083055460851716,en,2021-05-11T11:43:58
1,RT @DomainFQ: .\nDomain NameFor Sale\n\nhttps:...,1391353688401186818,1392083046459969541,en,2021-05-11T11:43:56
2,RT @DomainFQ: .\nDomains For sale\n\nhttps://t...,1391353349669154818,1392083045058940935,en,2021-05-11T11:43:56
3,RT @DomainFQ: .\nDomain NameFor Sale\n\nhttps:...,1391353688401186818,1392083027929358336,en,2021-05-11T11:43:52
5,RT @CharlesNkhwashu: Savanna Drinkers 🤦\n\n#R5...,1392030290256744448,1392082985189511168,en,2021-05-11T11:43:41
...,...,...,...,...,...
3390413,"RT @JenniferFaye34: ""You are gonna LOVE Leo!""😍...",1259825816600444929,1259832102637445120,en,2020-05-11T13:05:54
3390414,The currency future is upon us. #Crypto is her...,1259827939442601984,1259832076863451138,en,2020-05-11T13:05:48
3390415,RT @TechInRL: AirPods Pro User Guide and Tutor...,1259825608042872834,1259832072488792064,en,2020-05-11T13:05:47
3390416,RT @TechInRL: Apple Watch User Guide &amp; Tut...,1259802946377846784,1259832056277872641,en,2020-05-11T13:05:43


In [11]:
# print(isinstance(df1.head(10)['referenced_tweets'].values, float))
# df2['rt'] = ""
# df2.loc[isinstance(df2['referenced_tweets'].values, float) == False, 'rt'] = df2['referenced_tweets']

def text_iter4(ref_tweets,twid):
    dfcut.loc[twid == ref_tweets,'fulltext'] = df2.loc[twid == ref_tweets, 'text']
#     dfcut['fulltext'] = 
#     df2.loc[df2['id'] = x]
#     df2['fulltext'] = 
text_iter4(dfcut['referenced_tweets'].values,df2['id'].values)
dfcut
# for index, row in df1.head(10).iterrows():

#     print(np.array(df2['referenced_tweets'][index])[0]['id'])

<ipython-input-11-e0f892d7eb3e>:6: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  dfcut.loc[twid == ref_tweets,'fulltext'] = df2.loc[twid == ref_tweets, 'text']


KeyError: False

In [30]:
dftemp = pd.merge(left=dfcut,right=df2,how='left',left_on='referenced_tweets',right_on='id')
dftemp

,text_x,referenced_tweets_x,id_x,lang_x,created_at_x,text_y,referenced_tweets_y,id_y,lang_y,created_at_y
0,RT @DomainFQ: .\nDomain Name for sale\n\nhttps...,1391353131334717440,1392083055460851716,en,2021-05-11T11:43:58,.\nDomain Name for sale\n\nhttps://t.co/f056Wp...,NaN,1391353131334717440,en,2021-05-09T11:23:31
1,RT @DomainFQ: .\nDomain NameFor Sale\n\nhttps:...,1391353688401186818,1392083046459969541,en,2021-05-11T11:43:56,.\nDomain NameFor Sale\n\nhttps://t.co/a2N3rgJ...,NaN,1391353688401186818,en,2021-05-09T11:25:44
2,RT @DomainFQ: .\nDomains For sale\n\nhttps://t...,1391353349669154818,1392083045058940935,en,2021-05-11T11:43:56,.\nDomains For sale\n\nhttps://t.co/0YeRs3JJLb...,NaN,1391353349669154818,en,2021-05-09T11:24:23
3,RT @DomainFQ: .\nDomain NameFor Sale\n\nhttps:...,1391353688401186818,1392083027929358336,en,2021-05-11T11:43:52,.\nDomain NameFor Sale\n\nhttps://t.co/a2N3rgJ...,NaN,1391353688401186818,en,2021-05-09T11:25:44
4,RT @CharlesNkhwashu: Savanna Drinkers 🤦\n\n#R5...,1392030290256744448,1392082985189511168,en,2021-05-11T11:43:41,Savanna Drinkers 🤦\n\n#R500 #RHOA #AceMagashul...,NaN,1392030290256744448,en,2021-05-11T08:14:18
...,...,...,...,...,...,...,...,...,...,...
2033681,"RT @JenniferFaye34: ""You are gonna LOVE Leo!""😍...",1259825816600444929,1259832102637445120,en,2020-05-11T13:05:54,NaN,NaN,NaN,NaN,NaN
2033682,The currency future is upon us. #Crypto is her...,1259827939442601984,1259832076863451138,en,2020-05-11T13:05:48,NaN,NaN,NaN,NaN,NaN
2033683,RT @TechInRL: AirPods Pro User Guide and Tutor...,1259825608042872834,1259832072488792064,en,2020-05-11T13:05:47,NaN,NaN,NaN,NaN,NaN
2033684,RT @TechInRL: Apple Watch User Guide &amp; Tut...,1259802946377846784,1259832056277872641,en,2020-05-11T13:05:43,NaN,NaN,NaN,NaN,NaN


In [31]:
newtext = np.array([])
def textthing(x,y):
    for index in range(len(dftemp)):
        if y[index] == y[index]:
            x[index] = y[index]
    print(x.size)
#     dftemp['newtext'] = x
textthing(dftemp['text_x'].values,dftemp['text_y'].values)

dftemp

2033686


,text_x,referenced_tweets_x,id_x,lang_x,created_at_x,text_y,referenced_tweets_y,id_y,lang_y,created_at_y
0,.\nDomain Name for sale\n\nhttps://t.co/f056Wp...,1391353131334717440,1392083055460851716,en,2021-05-11T11:43:58,.\nDomain Name for sale\n\nhttps://t.co/f056Wp...,NaN,1391353131334717440,en,2021-05-09T11:23:31
1,.\nDomain NameFor Sale\n\nhttps://t.co/a2N3rgJ...,1391353688401186818,1392083046459969541,en,2021-05-11T11:43:56,.\nDomain NameFor Sale\n\nhttps://t.co/a2N3rgJ...,NaN,1391353688401186818,en,2021-05-09T11:25:44
2,.\nDomains For sale\n\nhttps://t.co/0YeRs3JJLb...,1391353349669154818,1392083045058940935,en,2021-05-11T11:43:56,.\nDomains For sale\n\nhttps://t.co/0YeRs3JJLb...,NaN,1391353349669154818,en,2021-05-09T11:24:23
3,.\nDomain NameFor Sale\n\nhttps://t.co/a2N3rgJ...,1391353688401186818,1392083027929358336,en,2021-05-11T11:43:52,.\nDomain NameFor Sale\n\nhttps://t.co/a2N3rgJ...,NaN,1391353688401186818,en,2021-05-09T11:25:44
4,Savanna Drinkers 🤦\n\n#R500 #RHOA #AceMagashul...,1392030290256744448,1392082985189511168,en,2021-05-11T11:43:41,Savanna Drinkers 🤦\n\n#R500 #RHOA #AceMagashul...,NaN,1392030290256744448,en,2021-05-11T08:14:18
...,...,...,...,...,...,...,...,...,...,...
2033681,"RT @JenniferFaye34: ""You are gonna LOVE Leo!""😍...",1259825816600444929,1259832102637445120,en,2020-05-11T13:05:54,NaN,NaN,NaN,NaN,NaN
2033682,The currency future is upon us. #Crypto is her...,1259827939442601984,1259832076863451138,en,2020-05-11T13:05:48,NaN,NaN,NaN,NaN,NaN
2033683,RT @TechInRL: AirPods Pro User Guide and Tutor...,1259825608042872834,1259832072488792064,en,2020-05-11T13:05:47,NaN,NaN,NaN,NaN,NaN
2033684,RT @TechInRL: Apple Watch User Guide &amp; Tut...,1259802946377846784,1259832056277872641,en,2020-05-11T13:05:43,NaN,NaN,NaN,NaN,NaN


In [32]:
dftemp2 = dftemp.iloc[: , :-5]
dftemp2.columns=["text", "referenced_tweets", "id", "lang","created_at"]
dftemp2

,text,referenced_tweets,id,lang,created_at
0,.\nDomain Name for sale\n\nhttps://t.co/f056Wp...,1391353131334717440,1392083055460851716,en,2021-05-11T11:43:58
1,.\nDomain NameFor Sale\n\nhttps://t.co/a2N3rgJ...,1391353688401186818,1392083046459969541,en,2021-05-11T11:43:56
2,.\nDomains For sale\n\nhttps://t.co/0YeRs3JJLb...,1391353349669154818,1392083045058940935,en,2021-05-11T11:43:56
3,.\nDomain NameFor Sale\n\nhttps://t.co/a2N3rgJ...,1391353688401186818,1392083027929358336,en,2021-05-11T11:43:52
4,Savanna Drinkers 🤦\n\n#R500 #RHOA #AceMagashul...,1392030290256744448,1392082985189511168,en,2021-05-11T11:43:41
...,...,...,...,...,...
2033681,"RT @JenniferFaye34: ""You are gonna LOVE Leo!""😍...",1259825816600444929,1259832102637445120,en,2020-05-11T13:05:54
2033682,The currency future is upon us. #Crypto is her...,1259827939442601984,1259832076863451138,en,2020-05-11T13:05:48
2033683,RT @TechInRL: AirPods Pro User Guide and Tutor...,1259825608042872834,1259832072488792064,en,2020-05-11T13:05:47
2033684,RT @TechInRL: Apple Watch User Guide &amp; Tut...,1259802946377846784,1259832056277872641,en,2020-05-11T13:05:43


In [33]:
dfother = df2
dfother = dfother[dfother['referenced_tweets'].isna()]
dfother.reset_index()
dffinal = pd.concat([dftemp2, dfother], ignore_index=True)
dffinal

,text,referenced_tweets,id,lang,created_at
0,.\nDomain Name for sale\n\nhttps://t.co/f056Wp...,1391353131334717440,1392083055460851716,en,2021-05-11T11:43:58
1,.\nDomain NameFor Sale\n\nhttps://t.co/a2N3rgJ...,1391353688401186818,1392083046459969541,en,2021-05-11T11:43:56
2,.\nDomains For sale\n\nhttps://t.co/0YeRs3JJLb...,1391353349669154818,1392083045058940935,en,2021-05-11T11:43:56
3,.\nDomain NameFor Sale\n\nhttps://t.co/a2N3rgJ...,1391353688401186818,1392083027929358336,en,2021-05-11T11:43:52
4,Savanna Drinkers 🤦\n\n#R500 #RHOA #AceMagashul...,1392030290256744448,1392082985189511168,en,2021-05-11T11:43:41
...,...,...,...,...,...
3390563,Silicone Case Cover Shockproof Holder For Appl...,NaN,1259830641736273921,en,2020-05-11T13:00:06
3390564,#Apple has shown that #iPhone sales fell more ...,NaN,1259830629723766785,en,2020-05-11T13:00:03
3390565,Check it OUT Shadowland by C.M.Gray Read the F...,NaN,1259830628721229826,en,2020-05-11T13:00:03
3390566,Le casque circum-aural d’Apple serait baptisé ...,NaN,1259830627270119427,fr,2020-05-11T13:00:02


In [45]:
# for index, row in df1.head(599999).iterrows():
# #     a = row['referenced_tweets']
#     if isinstance(df1['referenced_tweets'][index], float) == False:
#         rtweetid = df1['referenced_tweets'][index][0]['id']
# #         print(rtweetid)
#         if (df1.loc[df1['id'] == str(rtweetid)]).empty == False:
#             originalrow = df1.loc[df1['id'] == str(rtweetid)]
#             df2['text'][index] = originalrow.iloc[0,0]
# #         print(originalrow)
# #         print(originalrow.iloc[0,0])
# print(df1['referenced_tweets'].values[0])
# def text_iter3(ref_tweets,twid):
#     df2['fulltext'] = df1['text']
#     df2['rt'] = ""
#     if isinstance(ref_tweets, float) == False:
#         rtweetid = ref_tweets
#         if (df1.loc[twid == str(rtweetid)]).empty == False:
#             originalrow = df1.loc[twid == str(rtweetid)]
#             df2['fulltext'] = originalrow.iloc[0,0]

# df2['fulltext'] = text_iter3(df1['referenced_tweets'].values,df1['id'].values)

# df2.head(599999)

# df2['fulltext'] = df1['text']
# df2['fulltext']

# def soc_iter(TEAM,home,away,ftr):
#     #team, row['HomeTeam'], row['AwayTeam'], row['FTR']
#     if [((home == TEAM) & (ftr == 'D')) | ((away == TEAM) & (ftr == 'D'))]:
#         result = 'Draw'
#     elif [((home == TEAM) & (ftr != 'D')) | ((away == TEAM) & (ftr != 'D'))]:
#         result = 'No_Draw'
#     else:
#         result = 'No_Game'
#     return result

# def soc_iter(TEAM,home,away,ftr):
#     df['Draws'] = 'No_Game'
#     df.loc[((home == TEAM) & (ftr == 'D')) | ((away == TEAM) & (ftr == 'D')), 'Draws'] = 'Draw'
#     df.loc[((home == TEAM) & (ftr != 'D')) | ((away == TEAM) & (ftr != 'D')), 'Draws'] = 'No_Draw'

# def text_iter(ref_tweets,twid):
#     df2['fulltext'] = df1['text']
#     df2['rt'] = ""
#     df2.loc[isinstance(ref_tweets, float) == False, df2['rt']] = ref_tweets[0]['id']
#     df2.loc[(df2.loc[twid == str(rtweetid)]).empty == False, df2['fulltext']] = (df2.loc[df2['id'].values == str(rtweetid)]).iloc[0,0]

# def text_iter2(ref_tweets,twid):
#     df2['fulltext'] = df1['text']
#     df2['rt'] = ""
#     df2.loc[isinstance(ref_tweets, float) == False, 'rt'] = ref_tweets
#     df2.loc[twid == str(rtweetid).empty == False, 'fulltext'] = (df2.loc[twid == str(rtweetid)]).iloc[0,0]

    
# df2['fulltext'] = text_iter2(df2['referenced_tweets'].values,df2['id'].values)

[{'type': 'retweeted', 'id': '1391353131334717440'}]


IndexError: list index out of range

In [ ]:
# print(df1['referenced_tweets'][0][0]['id'])
# print(type(df1['id'][0]))

# for index, row in df1.iterrows():
# #     a = row['referenced_tweets']
#     if isinstance(df1['referenced_tweets'][index], float) == False:
#         rtweetid = df1['referenced_tweets'][index][0]['id']
#         originalrow = df1.loc[df1['id'] == rtweetid]
#         df1['text'][index] = originalrow['text']

In [60]:
# df1.to_csv("twitter_aapl_1_clean.csv", index = False)

In [34]:
dffinal2 = dffinal
# Only including this step for now, basically removes the time component, leaving only date
dffinal2['time'] = dffinal2['created_at'].map(lambda x: str(x)[11:])
dffinal2['created_at'] = dffinal2['created_at'].map(lambda x: str(x)[:-9])
dffinal2 = dffinal2.rename(columns={"created_at" : "Date"})
dffinal2

,text,referenced_tweets,id,lang,Date,time
0,.\nDomain Name for sale\n\nhttps://t.co/f056Wp...,1391353131334717440,1392083055460851716,en,2021-05-11,11:43:58
1,.\nDomain NameFor Sale\n\nhttps://t.co/a2N3rgJ...,1391353688401186818,1392083046459969541,en,2021-05-11,11:43:56
2,.\nDomains For sale\n\nhttps://t.co/0YeRs3JJLb...,1391353349669154818,1392083045058940935,en,2021-05-11,11:43:56
3,.\nDomain NameFor Sale\n\nhttps://t.co/a2N3rgJ...,1391353688401186818,1392083027929358336,en,2021-05-11,11:43:52
4,Savanna Drinkers 🤦\n\n#R500 #RHOA #AceMagashul...,1392030290256744448,1392082985189511168,en,2021-05-11,11:43:41
...,...,...,...,...,...,...
3390563,Silicone Case Cover Shockproof Holder For Appl...,NaN,1259830641736273921,en,2020-05-11,13:00:06
3390564,#Apple has shown that #iPhone sales fell more ...,NaN,1259830629723766785,en,2020-05-11,13:00:03
3390565,Check it OUT Shadowland by C.M.Gray Read the F...,NaN,1259830628721229826,en,2020-05-11,13:00:03
3390566,Le casque circum-aural d’Apple serait baptisé ...,NaN,1259830627270119427,fr,2020-05-11,13:00:02


In [34]:
# dffinal2.to_csv("dffinal2.csv", index = False)

dffinal2 = pd.read_csv('dffinal2.csv')

In [35]:
# adapted from https://www.30secondsofcode.org/python/s/is-weekend

def if_weekend(d = datetime.today()):
    if d == date(2021,4,2):
        return date(2021,4,5)
    elif d == date(2021,2,14):
        return date(2021,2,16)
    elif d == date(2021,2,13):
        return date(2021,2,16)    
    elif d == date(2021,2,15):#
        return date(2021,2,16)
    elif d == date(2021,1,16):#
        return date(2021,1,19) 
    elif d == date(2021,1,17):#
        return date(2021,1,19)    
    elif d == date(2021,1,18):#
        return date(2021,1,19)
    elif d == date(2021,1,1):
        return date(2021,1,4)
    elif d == date(2020,12,25):
        return date(2020,12,28)
    elif d == date(2020,11,26):
        return date(2020,11,27)
    elif d == date(2020,9,5):#
        return date(2020,9,8) 
    elif d == date(2020,9,6):#
        return date(2020,9,8)    
    elif d == date(2020,9,7):#
        return date(2020,9,8)
    elif d == date(2020,7,3):
        return date(2020,7,6)
    elif d == date(2020,5,23):#
        return date(2020,5,26)
    elif d == date(2020,5,24):#
        return date(2020,5,26)
    elif d == date(2020,5,25):#
        return date(2020,5,26)
    
    elif d.weekday() == 5:
        d = d + timedelta(days=2)
    elif d.weekday() == 6:
        d = d + timedelta(days=1)
    return d
def if_weekendmore(daysahead, d = datetime.today()):
    d = if_weekend(d)
    for day in range(daysahead):
        d = d + timedelta(days=1)
        if d.weekday() == 5:
            d = d + timedelta(days=2)
        elif d.weekday() == 6:
            d = d + timedelta(days=1)
        if d == date(2021,4,2):
            d = date(2021,4,5)
        elif d == date(2021,2,14):
            d = date(2021,2,16)
        elif d == date(2021,2,13):
            d = date(2021,2,16)    
        elif d == date(2021,2,15):#
            d = date(2021,2,16)
        elif d == date(2021,1,16):#
            d = date(2021,1,19) 
        elif d == date(2021,1,17):#
            d = date(2021,1,19)    
        elif d == date(2021,1,18):#
            d = date(2021,1,19)
        elif d == date(2021,1,1):
            d = date(2021,1,4)
        elif d == date(2020,12,25):
            d = date(2020,12,28)
        elif d == date(2020,11,26):
            d = date(2020,11,27)
        elif d == date(2020,9,5):#
            d = date(2020,9,8) 
        elif d == date(2020,9,6):#
            d = date(2020,9,8)    
        elif d == date(2020,9,7):#
            d = date(2020,9,8)
        elif d == date(2020,7,3):
            d = date(2020,7,6)
        elif d == date(2020,5,23):#
            d = date(2020,5,26)
        elif d == date(2020,5,24):#
            d = date(2020,5,26)
        elif d == date(2020,5,25):#
            d = date(2020,5,26)
    return d
    
print(if_weekend(date(2021,5,8)))
print(if_weekendmore(daysahead = 5, d = date(2021,5,11)))



print(type(date(2021,6,14)))

2021-05-10
2021-05-18
<class 'datetime.date'>


In [36]:
dffinal2['Date']= pd.to_datetime(dffinal2['Date'])
dffinal2['usedate'] = dffinal2['Date'].map(lambda x: if_weekend(x))
dffinal2['usedate1'] = dffinal2['Date'].map(lambda x: if_weekendmore(1,x))
dffinal2['usedate2'] = dffinal2['Date'].map(lambda x: if_weekendmore(2,x))
dffinal2['usedate3'] = dffinal2['Date'].map(lambda x: if_weekendmore(3,x))
dffinal2['usedate4'] = dffinal2['Date'].map(lambda x: if_weekendmore(4,x))
dffinal2['usedate5'] = dffinal2['Date'].map(lambda x: if_weekendmore(5,x))
dffinal2

,text,referenced_tweets,id,lang,Date,time,usedate,usedate1,usedate2,usedate3,usedate4,usedate5
0,.\nDomain Name for sale\n\nhttps://t.co/f056Wp...,1.391353e+18,1392083055460851716,en,2021-05-11,11:43:58,2021-05-11,2021-05-12,2021-05-13,2021-05-14,2021-05-17,2021-05-18
1,.\nDomain NameFor Sale\n\nhttps://t.co/a2N3rgJ...,1.391354e+18,1392083046459969541,en,2021-05-11,11:43:56,2021-05-11,2021-05-12,2021-05-13,2021-05-14,2021-05-17,2021-05-18
2,.\nDomains For sale\n\nhttps://t.co/0YeRs3JJLb...,1.391353e+18,1392083045058940935,en,2021-05-11,11:43:56,2021-05-11,2021-05-12,2021-05-13,2021-05-14,2021-05-17,2021-05-18
3,.\nDomain NameFor Sale\n\nhttps://t.co/a2N3rgJ...,1.391354e+18,1392083027929358336,en,2021-05-11,11:43:52,2021-05-11,2021-05-12,2021-05-13,2021-05-14,2021-05-17,2021-05-18
4,Savanna Drinkers 🤦\n\n#R500 #RHOA #AceMagashul...,1.392030e+18,1392082985189511168,en,2021-05-11,11:43:41,2021-05-11,2021-05-12,2021-05-13,2021-05-14,2021-05-17,2021-05-18
...,...,...,...,...,...,...,...,...,...,...,...,...
3390563,Silicone Case Cover Shockproof Holder For Appl...,NaN,1259830641736273921,en,2020-05-11,13:00:06,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-18
3390564,#Apple has shown that #iPhone sales fell more ...,NaN,1259830629723766785,en,2020-05-11,13:00:03,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-18
3390565,Check it OUT Shadowland by C.M.Gray Read the F...,NaN,1259830628721229826,en,2020-05-11,13:00:03,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-18
3390566,Le casque circum-aural d’Apple serait baptisé ...,NaN,1259830627270119427,fr,2020-05-11,13:00:02,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-18


In [37]:
dffinal3 = dffinal2
# dffinal3.to_csv("dffinal3.csv", index = False)

dffinal3 = pd.read_csv('dffinal3.csv')
dffinal3

,text,referenced_tweets,id,lang,Date,time,usedate,usedate1,usedate2,usedate3,usedate4,usedate5
0,.\nDomain Name for sale\n\nhttps://t.co/f056Wp...,1.391353e+18,1392083055460851716,en,2021-05-11,11:43:58,2021-05-11,2021-05-12,2021-05-13,2021-05-14,2021-05-17,2021-05-18
1,.\nDomain NameFor Sale\n\nhttps://t.co/a2N3rgJ...,1.391354e+18,1392083046459969541,en,2021-05-11,11:43:56,2021-05-11,2021-05-12,2021-05-13,2021-05-14,2021-05-17,2021-05-18
2,.\nDomains For sale\n\nhttps://t.co/0YeRs3JJLb...,1.391353e+18,1392083045058940935,en,2021-05-11,11:43:56,2021-05-11,2021-05-12,2021-05-13,2021-05-14,2021-05-17,2021-05-18
3,.\nDomain NameFor Sale\n\nhttps://t.co/a2N3rgJ...,1.391354e+18,1392083027929358336,en,2021-05-11,11:43:52,2021-05-11,2021-05-12,2021-05-13,2021-05-14,2021-05-17,2021-05-18
4,Savanna Drinkers 🤦\n\n#R500 #RHOA #AceMagashul...,1.392030e+18,1392082985189511168,en,2021-05-11,11:43:41,2021-05-11,2021-05-12,2021-05-13,2021-05-14,2021-05-17,2021-05-18
...,...,...,...,...,...,...,...,...,...,...,...,...
3390563,Silicone Case Cover Shockproof Holder For Appl...,NaN,1259830641736273921,en,2020-05-11,13:00:06,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-18
3390564,#Apple has shown that #iPhone sales fell more ...,NaN,1259830629723766785,en,2020-05-11,13:00:03,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-18
3390565,Check it OUT Shadowland by C.M.Gray Read the F...,NaN,1259830628721229826,en,2020-05-11,13:00:03,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-18
3390566,Le casque circum-aural d’Apple serait baptisé ...,NaN,1259830627270119427,fr,2020-05-11,13:00:02,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-18


In [38]:
price = pd.read_csv("aapl2.csv")
price['Date'] = price['Date'].map(lambda x: parser.parse(x).isoformat())
# Only including this step for now, basically removes the time component, leaving only date
price['Date'] = price['Date'].map(lambda x: str(x)[:-9])
price['Date']= pd.to_datetime(price['Date'])
# Change me if looking to change examination scope
price = price.rename(columns={"Date" : "usedate"})
price

,usedate,Close/Last,Volume,Open,High,Low,Change
0,2021-05-11,125.9100,126142800,123.5000,126.2700,122.7700,2.4100
1,2021-05-10,126.8500,88071230,129.4100,129.5400,126.8100,-2.5600
2,2021-05-07,130.2100,78973270,130.8500,131.2582,129.4750,-0.6400
3,2021-05-06,129.7400,78128330,127.8900,129.7500,127.1300,1.8500
4,2021-05-05,128.1000,84000900,129.2000,130.4500,127.9700,-1.1000
...,...,...,...,...,...,...,...
247,2020-05-18,78.7400,135372520,78.2925,79.1250,77.5810,0.4475
248,2020-05-15,76.9275,166348360,75.0875,76.9750,75.0525,1.8400
249,2020-05-14,77.3850,158929080,76.1275,77.4475,75.3825,1.2575
250,2020-05-13,76.9125,200622560,78.0375,78.9875,75.8025,-1.1250


In [45]:
merged = pd.merge(dffinal2, price, how='left', on='usedate')
merged

,text,referenced_tweets,id,lang,Date,time,usedate,usedate1,usedate2,usedate3,usedate4,usedate5,Close/Last,Volume,Open,High,Low,Change
0,.\nDomain Name for sale\n\nhttps://t.co/f056Wp...,1.391353e+18,1392083055460851716,en,2021-05-11,11:43:58,2021-05-11,2021-05-12,2021-05-13,2021-05-14,2021-05-17,2021-05-18,125.91,126142800.0,123.5,126.27,122.77,2.41
1,.\nDomain NameFor Sale\n\nhttps://t.co/a2N3rgJ...,1.391354e+18,1392083046459969541,en,2021-05-11,11:43:56,2021-05-11,2021-05-12,2021-05-13,2021-05-14,2021-05-17,2021-05-18,125.91,126142800.0,123.5,126.27,122.77,2.41
2,.\nDomains For sale\n\nhttps://t.co/0YeRs3JJLb...,1.391353e+18,1392083045058940935,en,2021-05-11,11:43:56,2021-05-11,2021-05-12,2021-05-13,2021-05-14,2021-05-17,2021-05-18,125.91,126142800.0,123.5,126.27,122.77,2.41
3,.\nDomain NameFor Sale\n\nhttps://t.co/a2N3rgJ...,1.391354e+18,1392083027929358336,en,2021-05-11,11:43:52,2021-05-11,2021-05-12,2021-05-13,2021-05-14,2021-05-17,2021-05-18,125.91,126142800.0,123.5,126.27,122.77,2.41
4,Savanna Drinkers 🤦\n\n#R500 #RHOA #AceMagashul...,1.392030e+18,1392082985189511168,en,2021-05-11,11:43:41,2021-05-11,2021-05-12,2021-05-13,2021-05-14,2021-05-17,2021-05-18,125.91,126142800.0,123.5,126.27,122.77,2.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3390563,Silicone Case Cover Shockproof Holder For Appl...,NaN,1259830641736273921,en,2020-05-11,13:00:06,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-18,NaN,NaN,NaN,NaN,NaN,NaN
3390564,#Apple has shown that #iPhone sales fell more ...,NaN,1259830629723766785,en,2020-05-11,13:00:03,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-18,NaN,NaN,NaN,NaN,NaN,NaN
3390565,Check it OUT Shadowland by C.M.Gray Read the F...,NaN,1259830628721229826,en,2020-05-11,13:00:03,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-18,NaN,NaN,NaN,NaN,NaN,NaN
3390566,Le casque circum-aural d’Apple serait baptisé ...,NaN,1259830627270119427,fr,2020-05-11,13:00:02,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-18,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
pricesmall = price[['usedate','Change']]

pricesmall = pricesmall.rename(columns={"usedate" : "usedate1", "Change" : "Change1"})
merged = pd.merge(merged, pricesmall, how='left', on='usedate1')
pricesmall = pricesmall.rename(columns={"usedate1" : "usedate2", "Change1" : "Change2"})
merged = pd.merge(merged, pricesmall, how='left', on='usedate2')
pricesmall = pricesmall.rename(columns={"usedate2" : "usedate3", "Change2" : "Change3"})
merged = pd.merge(merged, pricesmall, how='left', on='usedate3')
pricesmall = pricesmall.rename(columns={"usedate3" : "usedate4", "Change3" : "Change4"})
merged = pd.merge(merged, pricesmall, how='left', on='usedate4')
pricesmall = pricesmall.rename(columns={"usedate4" : "usedate5", "Change4" : "Change5"})
merged = pd.merge(merged, pricesmall, how='left', on='usedate5')
merged
# def datematch(x,y,z):
#     for index in range(len(merged)):
#         if y[index] == x[index]:
#             return z[index]

# merged['Change1'] = price['Change'].loc[price['usedate'] == merged['usedate1']]
# merged

,text,referenced_tweets,id,lang,Date,time,usedate,usedate1,usedate2,usedate3,...,Volume,Open,High,Low,Change,Change1,Change2,Change3,Change4,Change5
0,.\nDomain Name for sale\n\nhttps://t.co/f056Wp...,1.391353e+18,1392083055460851716,en,2021-05-11,11:43:58,2021-05-11,2021-05-12,2021-05-13,2021-05-14,...,126142800.0,123.5,126.27,122.77,2.41,NaN,NaN,NaN,NaN,NaN
1,.\nDomain NameFor Sale\n\nhttps://t.co/a2N3rgJ...,1.391354e+18,1392083046459969541,en,2021-05-11,11:43:56,2021-05-11,2021-05-12,2021-05-13,2021-05-14,...,126142800.0,123.5,126.27,122.77,2.41,NaN,NaN,NaN,NaN,NaN
2,.\nDomains For sale\n\nhttps://t.co/0YeRs3JJLb...,1.391353e+18,1392083045058940935,en,2021-05-11,11:43:56,2021-05-11,2021-05-12,2021-05-13,2021-05-14,...,126142800.0,123.5,126.27,122.77,2.41,NaN,NaN,NaN,NaN,NaN
3,.\nDomain NameFor Sale\n\nhttps://t.co/a2N3rgJ...,1.391354e+18,1392083027929358336,en,2021-05-11,11:43:52,2021-05-11,2021-05-12,2021-05-13,2021-05-14,...,126142800.0,123.5,126.27,122.77,2.41,NaN,NaN,NaN,NaN,NaN
4,Savanna Drinkers 🤦\n\n#R500 #RHOA #AceMagashul...,1.392030e+18,1392082985189511168,en,2021-05-11,11:43:41,2021-05-11,2021-05-12,2021-05-13,2021-05-14,...,126142800.0,123.5,126.27,122.77,2.41,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3390563,Silicone Case Cover Shockproof Holder For Appl...,NaN,1259830641736273921,en,2020-05-11,13:00:06,2020-05-11,2020-05-12,2020-05-13,2020-05-14,...,NaN,NaN,NaN,NaN,NaN,-1.605,-1.125,1.2575,1.84,0.4475
3390564,#Apple has shown that #iPhone sales fell more ...,NaN,1259830629723766785,en,2020-05-11,13:00:03,2020-05-11,2020-05-12,2020-05-13,2020-05-14,...,NaN,NaN,NaN,NaN,NaN,-1.605,-1.125,1.2575,1.84,0.4475
3390565,Check it OUT Shadowland by C.M.Gray Read the F...,NaN,1259830628721229826,en,2020-05-11,13:00:03,2020-05-11,2020-05-12,2020-05-13,2020-05-14,...,NaN,NaN,NaN,NaN,NaN,-1.605,-1.125,1.2575,1.84,0.4475
3390566,Le casque circum-aural d’Apple serait baptisé ...,NaN,1259830627270119427,fr,2020-05-11,13:00:02,2020-05-11,2020-05-12,2020-05-13,2020-05-14,...,NaN,NaN,NaN,NaN,NaN,-1.605,-1.125,1.2575,1.84,0.4475


In [47]:
print(len(merged['usedate'].unique()))

dfwhatisgoingon = merged
dfwhatisgoingon = dfwhatisgoingon[dfwhatisgoingon['Volume'].isna()]
print(dfwhatisgoingon['usedate'].unique())
print(dfwhatisgoingon['Date'].unique())
dfwhatisgoingon

253
['2020-05-11T00:00:00.000000000']
['2020-05-11T00:00:00.000000000']


,text,referenced_tweets,id,lang,Date,time,usedate,usedate1,usedate2,usedate3,...,Volume,Open,High,Low,Change,Change1,Change2,Change3,Change4,Change5
2032740,🚨 NEW EPISODE ALERT 🚨\n\nTensions were HIGH! \...,1.259994e+18,1259996579949314049,en,2020-05-11,23:59:28,2020-05-11,2020-05-12,2020-05-13,2020-05-14,...,NaN,NaN,NaN,NaN,NaN,-1.605,-1.125,1.2575,1.84,0.4475
2032741,USE ESSES 6 TWEAKS PARA EVITAR PROBLEMAS COM O...,1.259992e+18,1259995145061011457,es,2020-05-11,23:53:46,2020-05-11,2020-05-12,2020-05-13,2020-05-14,...,NaN,NaN,NaN,NaN,NaN,-1.605,-1.125,1.2575,1.84,0.4475
2032742,RT @MacStadium: Need to access your Mac that's...,1.256630e+18,1259994892245168128,en,2020-05-11,23:52:46,2020-05-11,2020-05-12,2020-05-13,2020-05-14,...,NaN,NaN,NaN,NaN,NaN,-1.605,-1.125,1.2575,1.84,0.4475
2032743,News:\n\nOut of 38 companies including @Google...,1.259983e+18,1259994322096861187,en,2020-05-11,23:50:30,2020-05-11,2020-05-12,2020-05-13,2020-05-14,...,NaN,NaN,NaN,NaN,NaN,-1.605,-1.125,1.2575,1.84,0.4475
2032744,RT @rshevlin: #Google: The Next Big #Fintech V...,1.259821e+18,1259993575795953671,en,2020-05-11,23:47:32,2020-05-11,2020-05-12,2020-05-13,2020-05-14,...,NaN,NaN,NaN,NaN,NaN,-1.605,-1.125,1.2575,1.84,0.4475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3390563,Silicone Case Cover Shockproof Holder For Appl...,NaN,1259830641736273921,en,2020-05-11,13:00:06,2020-05-11,2020-05-12,2020-05-13,2020-05-14,...,NaN,NaN,NaN,NaN,NaN,-1.605,-1.125,1.2575,1.84,0.4475
3390564,#Apple has shown that #iPhone sales fell more ...,NaN,1259830629723766785,en,2020-05-11,13:00:03,2020-05-11,2020-05-12,2020-05-13,2020-05-14,...,NaN,NaN,NaN,NaN,NaN,-1.605,-1.125,1.2575,1.84,0.4475
3390565,Check it OUT Shadowland by C.M.Gray Read the F...,NaN,1259830628721229826,en,2020-05-11,13:00:03,2020-05-11,2020-05-12,2020-05-13,2020-05-14,...,NaN,NaN,NaN,NaN,NaN,-1.605,-1.125,1.2575,1.84,0.4475
3390566,Le casque circum-aural d’Apple serait baptisé ...,NaN,1259830627270119427,fr,2020-05-11,13:00:02,2020-05-11,2020-05-12,2020-05-13,2020-05-14,...,NaN,NaN,NaN,NaN,NaN,-1.605,-1.125,1.2575,1.84,0.4475


In [48]:
subset_df = merged[merged["usedate"] == '2020-05-11']
column_count = subset_df.count()
print(column_count)

text                 2692
referenced_tweets     946
id                   2692
lang                 2692
Date                 2692
time                 2692
usedate              2692
usedate1             2692
usedate2             2692
usedate3             2692
usedate4             2692
usedate5             2692
Close/Last              0
Volume                  0
Open                    0
High                    0
Low                     0
Change                  0
Change1              2692
Change2              2692
Change3              2692
Change4              2692
Change5              2692
dtype: int64


In [49]:
merged.isna().sum()

text                       0
referenced_tweets    1356882
id                         0
lang                       0
Date                       0
time                       0
usedate                    0
usedate1                   0
usedate2                   0
usedate3                   0
usedate4                   0
usedate5                   0
Close/Last              2692
Volume                  2692
Open                    2692
High                    2692
Low                     2692
Change                  2692
Change1                 3142
Change2                23221
Change3                29977
Change4                38182
Change5                45652
dtype: int64

In [50]:

merged = merged[merged['usedate'] != '2020-05-11']
merged

,text,referenced_tweets,id,lang,Date,time,usedate,usedate1,usedate2,usedate3,...,Volume,Open,High,Low,Change,Change1,Change2,Change3,Change4,Change5
0,.\nDomain Name for sale\n\nhttps://t.co/f056Wp...,1.391353e+18,1392083055460851716,en,2021-05-11,11:43:58,2021-05-11,2021-05-12,2021-05-13,2021-05-14,...,126142800.0,123.5000,126.270,122.7700,2.410,NaN,NaN,NaN,NaN,NaN
1,.\nDomain NameFor Sale\n\nhttps://t.co/a2N3rgJ...,1.391354e+18,1392083046459969541,en,2021-05-11,11:43:56,2021-05-11,2021-05-12,2021-05-13,2021-05-14,...,126142800.0,123.5000,126.270,122.7700,2.410,NaN,NaN,NaN,NaN,NaN
2,.\nDomains For sale\n\nhttps://t.co/0YeRs3JJLb...,1.391353e+18,1392083045058940935,en,2021-05-11,11:43:56,2021-05-11,2021-05-12,2021-05-13,2021-05-14,...,126142800.0,123.5000,126.270,122.7700,2.410,NaN,NaN,NaN,NaN,NaN
3,.\nDomain NameFor Sale\n\nhttps://t.co/a2N3rgJ...,1.391354e+18,1392083027929358336,en,2021-05-11,11:43:52,2021-05-11,2021-05-12,2021-05-13,2021-05-14,...,126142800.0,123.5000,126.270,122.7700,2.410,NaN,NaN,NaN,NaN,NaN
4,Savanna Drinkers 🤦\n\n#R500 #RHOA #AceMagashul...,1.392030e+18,1392082985189511168,en,2021-05-11,11:43:41,2021-05-11,2021-05-12,2021-05-13,2021-05-14,...,126142800.0,123.5000,126.270,122.7700,2.410,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3388817,"Download JOBIN THE PENGUIN! Awesome, retro-ins...",NaN,1259996823118319621,en,2020-05-12,00:00:26,2020-05-12,2020-05-13,2020-05-14,2020-05-15,...,162301040.0,79.4575,79.922,77.7275,-1.605,-1.125,1.2575,1.84,0.4475,-0.4725
3388818,"2019 13"" 1.6GHz/128GB #Apple #MacBook Airs, re...",NaN,1259996818450132993,en,2020-05-12,00:00:25,2020-05-12,2020-05-13,2020-05-14,2020-05-15,...,162301040.0,79.4575,79.922,77.7275,-1.605,-1.125,1.2575,1.84,0.4475,-0.4725
3388819,How to use Apple Maps mobility trends data htt...,NaN,1259996729451085827,en,2020-05-12,00:00:04,2020-05-12,2020-05-13,2020-05-14,2020-05-15,...,162301040.0,79.4575,79.922,77.7275,-1.605,-1.125,1.2575,1.84,0.4475,-0.4725
3388820,AirPods Studio Could Come With Head And Neck D...,NaN,1259996725638504448,en,2020-05-12,00:00:03,2020-05-12,2020-05-13,2020-05-14,2020-05-15,...,162301040.0,79.4575,79.922,77.7275,-1.605,-1.125,1.2575,1.84,0.4475,-0.4725


In [51]:
merged.to_csv("mergedfullwithmoredates.csv", index = False)